In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pyspark-sample-dataset/sample_data.csv


In [2]:
!pip install findspark
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s              
  Preparing metadata (setup.py) ... - \ done
     |████████████████████████████████| 198 kB 38.2 MB/s            
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2d3405ff9f90b0a1a7dce471e575bd5b519c3a68484f21b298d7b901fae47693
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.4
    Uninstalling py4j-0.10.9.4:
      Successfully uninstalled py4j-0.10.9.4


In [3]:
import findspark
findspark.init()
import pyspark

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('data_processing').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 07:43:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.sql.types import StringType, DoubleType

In [6]:
df = spark.read.csv('../input/pyspark-sample-dataset', inferSchema=True, header=True)
df.show()

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
|      4| 37|      16.5|     4|  Apple|
|      5| 27|       9.0|     1|     MI|
|      4| 27|       9.0|     0|   Oppo|
|      5| 37|      23.0|     5|   Vivo|
|      5| 37|      23.0|     5|Samsung|
|      3| 22|       2.5|     0|  Apple|
|      3| 27|       6.0|     0|     MI|
|      2| 27|       6.0|     2|   Oppo|
|      5| 27|       6.0|     2|Samsung|
|      3| 37|      16.5|     5|  Apple|
|      5| 27|       6.0|     0|     MI|
|      4| 22|       6.0|     1|   Oppo|
|      4| 37|       9.0|     2|Samsung|
|      4| 27|       6.0|     1|  Apple|
|      1| 37|      23.0|     5|     MI|
|      2| 42|      23.0|     2|   Oppo|
|      4| 37|       6.0|     0|   Vivo|
+-------+---+----------+------+-------+
only showing top 20 rows



In [7]:
#check the columns
df.columns

['ratings', 'age', 'experience', 'family', 'mobile']

In [8]:
# number of columns
len(df.columns)

5

In [9]:
#no of rows
df.count()

33

In [10]:
#since in pyspark, there is no direct method to get the shape, we use the combination of the two,
print('The shape of the dataframe is:({},{})'.format((df.count()),(len(df.columns))))

The shape of the dataframe is:(33,5)


In [11]:
#the datatypes of the columns are as follows:
df.printSchema()

root
 |-- ratings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: double (nullable = true)
 |-- family: integer (nullable = true)
 |-- mobile: string (nullable = true)



In [12]:
#to display the first 5 rows
df.show(5)

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
|      4| 37|      16.5|     4|  Apple|
|      5| 27|       9.0|     1|     MI|
+-------+---+----------+------+-------+
only showing top 5 rows



In [13]:
#lets check only the 2 columns age & mobile, see only 5 rows
df.select('age', 'mobile').show(5)

+---+-------+
|age| mobile|
+---+-------+
| 32|   Vivo|
| 27|  Apple|
| 22|Samsung|
| 37|  Apple|
| 27|     MI|
+---+-------+
only showing top 5 rows



In [14]:
#summary stats
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------+
|summary|           ratings|               age|        experience|            family|mobile|
+-------+------------------+------------------+------------------+------------------+------+
|  count|                33|                33|                33|                33|    33|
|   mean|3.5757575757575757|30.484848484848484|10.303030303030303|1.8181818181818181|  null|
| stddev|1.1188806636071336|  6.18527087180309| 6.770731351213326|1.8448330794164254|  null|
|    min|                 1|                22|               2.5|                 0| Apple|
|    max|                 5|                42|              23.0|                 5|  Vivo|
+-------+------------------+------------------+------------------+------------------+------+



In [15]:
#add a new columns, which is the current column plus 10
df = df.withColumn('age_after_10_years', (df['age']+10))
df.show(10, False)

+-------+---+----------+------+-------+------------------+
|ratings|age|experience|family|mobile |age_after_10_years|
+-------+---+----------+------+-------+------------------+
|3      |32 |9.0       |3     |Vivo   |42                |
|3      |27 |13.0      |3     |Apple  |37                |
|4      |22 |2.5       |0     |Samsung|32                |
|4      |37 |16.5      |4     |Apple  |47                |
|5      |27 |9.0       |1     |MI     |37                |
|4      |27 |9.0       |0     |Oppo   |37                |
|5      |37 |23.0      |5     |Vivo   |47                |
|5      |37 |23.0      |5     |Samsung|47                |
|3      |22 |2.5       |0     |Apple  |32                |
|3      |27 |6.0       |0     |MI     |37                |
+-------+---+----------+------+-------+------------------+
only showing top 10 rows



In [16]:
#change the columns age dtype to double

